In [3]:
import numpy as np
# import pandas as pd
# import yfinance

from scipy.stats import norm

A Demonstration of Black-Scholes Pricing Formulation.

Only Requires the Following:
    - Stock Price, S 
    - Strike Price, K 
    - Risk-free Rate (Interest Rate), r
    - Time to Expiry (in Years), t
    - Volatility, $\sigma$



Definitions:

1) Brownian Motion at time $\it{t}$ as $\it{B_{\it{t}}}$

2) $\it{B_{0}} := 0$ which is the standard Brownian Motion assumption

3) $\it{B_{t}} - \it{B_{s}} := \it{N}(0,t-s)$ for t > s, which is also a standard Brownian Motion assumption

4) Derivative of $\it{B_{\it{t}}}$ is $d\it{B_{\it{t}}}$. The integral of $d\it{B_{\it{t}}}$ is $\it{B_{\it{t}}}$

TODO: Fix this Lemma Part
Ito's Lemma
$$
% \begin{aligned}
% \f{\it{B}}

df(X_t, t) = \frac{\partial f}{\partial t} dt + \frac{\partial f}{\partial x} dX_t + \frac{1}{2} \frac{\partial^2 f}{\partial x^2} (dX_t)^2 \\
= \left(\frac{\partial f}{\partial t} + \mu(X_t, t) \frac{\partial f}{\partial x} + \frac{1}{2} \sigma^2(X_t, t) \frac{\partial^2 f}{\partial x^2}\right) dt + \sigma(X_t, t) \frac{\partial f}{\partial x} dW_t

\\

\\

\\

f(X_t, t) - f(X_0, 0) = \int_0^t \frac{\partial f}{\partial t}(X_s, s) ds + \int_0^t \frac{\partial f}{\partial x}(X_s, s) dX_s + \frac{1}{2} \int_0^t \frac{\partial^2 f}{\partial x^2}(X_s, s) d\langle X \rangle_s

% f(\it{B_{t}}) = f(\it{B_{0}}) + \int_{0}^{t} f'(B_{s}) d\! x
% \end{aligned}
$$



Black Scholes Formula

For some call option

The closed form solution of a $\bold{European}$ call option is as follows:

For some current price $ S $, strike price $ X $, risk-free rate (annualized) $ r $, time to expiry in years $ t $, volatility $ \sigma $,

The European Call Price $ C $, and Put Price $ P $ can be priced at: 

$$

h^\pm = \frac{\ln\left(\frac{S \cdot (1 + r)^t}{X}\right) \pm 0.5 \cdot \sigma^2 \cdot t}{\sigma \cdot \sqrt{t}}

\\

C = S \cdot \Phi(h^{+}) - \frac{X}{(1 + r)^t} \cdot \Phi(h^{-})

\\

P = \frac{X}{(1 + r)^t} \cdot \Phi(-h^{-}) - S \cdot \Phi(-h^{+})

\\

$$


In [4]:

# Sample Values for SPY Option Price. 23 Jul
S = 553.7800 # Trading Price of instrument
K = 555.0000 # Exercise Price (Strike Price)
r = 0.04225 # Risk-free interest rate (Annualized)
t = 3/365 # Time to Expiry
sigma = 0.1 # Volatility (Annualized)



q = 0.01 # Dividend Yield (Annualized)
C = 0 # Call Option Price
P = 0 # Put Option Price

In [5]:
# C = S*np.exp(-q*t)*norm.cdf(x=(np.log(S/X)+(r-q+0.5*sigma**2)*t)/(sigma*np.sqrt(t)))

d1 = (np.log(S/K) + ((r-q) + 0.5*sigma**2)*t) / (sigma*np.sqrt(t))
d2 = d1 - sigma*(np.sqrt(t))

C = norm.cdf(d1)*np.exp(-q*t)*S - norm.cdf(d2)*np.exp(-r*t)*K
P = norm.cdf(-d2)*np.exp(-r*t)*K - norm.cdf(-d1)*np.exp(-q*t)*S


In [6]:
d1,d2

(np.float64(-0.2089631620829448), np.float64(-0.2180291303611773))

In [7]:
print(f"d1: {d1}; d2:{d2}; \n \
    Call Option Price: {C}; Put Option Price: {P}")

d1: -0.2089631620829448; d2:-0.2180291303611773; 
     Call Option Price: 1.5137840421169244; Put Option Price: 2.5866023437375816


# Black-76 Model

Priced with fair strike, $F$ instead of dividend yield, $q$.
This is primarily because estimating dividend yield, is often difficult.

In [8]:
F = S*np.exp((r-q)*t) # Forward Price or the fair strike price

d1_76 = (np.log(F/K) + 0.5*sigma**2*t) / (sigma*np.sqrt(t))
d2_76 = d1_76 - sigma*(np.sqrt(t))

C_76 = np.exp(-r * t) * (norm.cdf(d1_76) * F - norm.cdf(d2_76) * K)
P_76 = np.exp(-r * t) * (norm.cdf(-d2_76) * K - norm.cdf(-d1_76) * F)

# Supplementary notes: 
# norm.cdf(d1_76) * F ; Undiscounted Exposure to S 
# multiplying by exp(-r*t) gives the discounted exposure to S; this accounts for the time value of money

# norm.cdf(d2_76); This is the risk-neutral probability of S > K at expiry

In [9]:
print(f"d1: {d1_76}; d2:{d2_76}; \n \
    Call Option Price: {C_76}; Put Option Price: {P_76}")

d1: -0.20896316208294774; d2:-0.21802913036118024; 
     Call Option Price: 1.5137840421169475; Put Option Price: 2.5866023437376455


# Implied Probability

Going from Price of Call/Put -> Greeks/Parameters

In [10]:
# P(K) is the price of a put option at strike K at time t.

#Q(S < K) = np.exp(-r*t) * dP(K) / dK 

Q_S_K = norm.cdf(-d2) # Probability of S < K at expiry; using d2_76 also works
Q_dS = norm.cdf(d2) / (K * sigma * np.sqrt(t))  # *dK for some small change in K. 


# Greeks


## Greeks: Delta

1. Delta, $\triangle$, defined as $ \frac{\partial C}{\partial S} $ <br>
1.1. A proxy uses the forward price, $F$, 
<!-- $$
\triangle _{t} = \exp(-r*t) \frac{\partial C}{\partial F}
$$ -->

2. Delta is stable across expires for fixed, normalized log-moneyness
3. Cash Delta = $F \triangle$

In [11]:
C_delta = np.exp(-r*t) * norm.cdf(d1)
P_delta = -np.exp(-r*t) * norm.cdf(-d1)

Cash_delta = F * C_delta # Cash delta is the amount of cash needed to hedge the option position.
# Cash delta = F * P_delta # Cash delta is the amount of cash needed to hedge the option position.

C_delta, P_delta

(np.float64(0.4170936328729004), np.float64(-0.5825591671409971))

In [12]:
# Show how delta is stable across expires
# F_example = 100
# sigma_example = 0.2
# r_example = 0.05

## Greeks: Gamma

1. Gamma, $\gamma$, defined as $ \frac{\partial^2 C}{\partial S^2} $ <br>
   1. A proxy uses the forward price, $F$,
   $$ 
   \Gamma _{t} = \exp(-r*t) \frac{\phi (d_{1})}{F \sigma \sqrt{t}}
   $$
2. Cash Gamma, $\Gamma _{cash,t} = \Gamma _{t} F^{2}$
   1. Cash gamma is proportional to derivative of option price wrt variance
   $$
   \frac{1}{2}\$ \Gamma = \frac{\partial C}{\partial [\sigma^2 t]} 
   $$


In [13]:
Gamma = np.exp(-r*t) * norm.pdf(d1) / (F * sigma * np.sqrt(t)) # Gamma is the rate of change of delta with respect to the underlying asset price.
Cash_gamma = F ** 2 * Gamma 



## Greeks: Theta

1. Theta, $\theta$, defined as $ \frac{\partial C}{\partial t} $ <br>
2. 2 Primary Components: Interest Rate & Volatility
   1. Formula : $ \Omega = r\it{X}e^{-rt} + e^{-rt}\frac{\partial \it{X}}{\partial t}$
   2. First term is $\Omega _{r}$
   3. Second term is $\Omega _{t}$
   4. $ \it{X} = e^{rt} C $; which is the undiscounted price of option
3. $ 0.5 \$\Gamma \sigma^{2} + \Omega _{t} = 0 $ 

In [14]:
Xt = np.exp(r * t) * C

Theta_r = np.exp(-r * t) * Xt 
# Theta_sigma = np.exp(-r * t) * delX/ delT # Need to approximate delX/delT, so this method is not used for now.
Theta_sigma = -0.5 * Cash_gamma * sigma**2

Theta = Theta_r + Theta_sigma

Theta

np.float64(-117.68877565001432)